In [40]:
prompt_content = open('prompts/scientific_papers_prompt.txt', encoding='utf-8').read()
prompt_content

'科学研究论文：\n--- \n{document} \n---\n\n您是分析科学研究论文的专家。 请仔细阅读上面提供的研究论文，并提取以下关键信息：\n\n从研究论文中提取以下六 (6) 个属性：\n- 论文标题：研究论文的全名\n- 出版年份：论文发表的年份\n- 作者：论文所有作者的全名\n- 作者联系方式：字典列表，其中每个字典包含每个作者的以下键：\n  - 姓名：作者的全名\n  - 机构：作者的机构隶属关系\n  - 电子邮件：作者的电子邮件地址（如果提供）\n- 摘要：论文摘要的全文\n- 摘要总结：用 2-3 句话简洁地总结摘要，突出重点，此字段为中文。\n\n指南：\n- 提取的信息应属实，并准确无误。\n- 除摘要外，应极其简洁，摘要应完整复制。\n- 提取的实体应该是独立的，并且不需要论文的其余部分就能轻松理解。\n- 如果论文中缺少任何属性，请将该字段留空，而不是猜测。\n- 对于摘要总结，重点介绍研究的主要目标、方法和主要发现。\n- 对于作者联系方式，请为每个作者创建一个条目，即使缺少一些信息。如果没有提供作者的电子邮件或机构，请在字典中将该字段留空。\n\n以 JSON 格式回答。 JSON 应包含 6 个键："PaperTitle", "PublicationYear", "Authors", "AuthorContact", "Abstract", "SummaryAbstract"。 "AuthorContact"字段应该是字典列表格式。'

In [41]:
import cntext as ct

paper_content = ct.read_pdf('data/1706.03762v7.pdf')[:4000]
paper_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and con

In [32]:
%%time

import ollama
import cntext as ct  #cntext版本2.1.3

paper_content = ct.read_pdf('data/1706.03762v7.pdf')[:4000]
prompt_content = open('prompts/scientific_papers_prompt.txt', encoding='utf-8').read()

response = ollama.chat(model='llama3.1:8b', 
                       messages = [
                           {'role': 'system', 'content': prompt_content},
                           {'role': 'user', 'content': paper_content}
                       ])

result = response['message']['content']
result = eval(result.split('```\n')[1].split('\n```')[0])
result

{'PaperTitle': 'Attention Is All You Need',
 'PublicationYear': 2017,
 'Authors': ['Ashish Vaswani',
  'Noam Shazeer',
  'Niki Parmar',
  'Jakob Uszkoreit',
  'Llion Jones',
  'Aidan N. Gomez',
  'Łukasz Kaiser',
  'Illia Polosukhin'],
 'AuthorContact': [{'Name': 'Ashish Vaswani',
   'Institution': 'Google Brain',
   'Email': 'avaswani@google.com'},
  {'Name': 'Noam Shazeer',
   'Institution': 'Google Brain',
   'Email': 'noam@google.com'},
  {'Name': 'Niki Parmar',
   'Institution': 'Google Research',
   'Email': 'nikip@google.com'},
  {'Name': 'Jakob Uszkoreit',
   'Institution': 'Google Research',
   'Email': 'usz@google.com'},
  {'Name': 'Llion Jones',
   'Institution': 'Google Research',
   'Email': 'llion@google.com'},
  {'Name': 'Aidan N. Gomez',
   'Institution': 'University of Toronto',
   'Email': 'aidan@cs.toronto.edu'},
  {'Name': 'Łukasz Kaiser',
   'Institution': 'Google Brain',
   'Email': 'lukaszkaiser@google.com'},
  {'Name': 'Illia Polosukhin',
   'Institution': '',
 

In [43]:
import ollama
import cntext as ct  
#cntext版本为2.1.2，非开源， #需联系大邓372335839获取

def extract_info(paper_content, prompt_content, max_retries=3):
    for attempt in range(max_retries + 1):
        try:
            response = ollama.chat(
                model='llama3.1:8b',
                messages=[
                    {'role': 'system', 'content': prompt_content},
                    {'role': 'user', 'content': paper_content}
                ]
            )

            result = response['message']['content']
            result = eval(result.split('```\n')[1].split('\n```')[0])
            return result
        
        except Exception as e:
            if attempt < max_retries:
                print(f"An error occurred: {e}. Retrying ({attempt + 1}/{max_retries + 1})...")
            else:
                raise e


#我们感兴趣的信息在论文的第一页，所以这里粗糙的选择前4000个字符。
paper_content = ct.read_pdf('data/1706.03762v7.pdf')[:4000]
prompt_content = open('prompts/scientific_papers_prompt.txt', encoding='utf-8').read()

result = extract_info(paper_content, prompt_content)
result

{'PaperTitle': 'Attention Is All You Need',
 'PublicationYear': 2017,
 'Authors': [{'Name': 'Ashish Vaswani', 'Institution': 'Google Brain'},
  {'Name': 'Noam Shazeer', 'Institution': 'Google Brain'},
  {'Name': 'Niki Parmar', 'Institution': 'Google Research'},
  {'Name': 'Jakob Uszkoreit', 'Institution': 'Google Research'},
  {'Name': 'Llion Jones', 'Institution': 'Google Research'},
  {'Name': 'Aidan N. Gomez',
   'Institution': 'University of Toronto',
   'Email': 'aidan@cs.toronto.edu'},
  {'Name': 'Łukasz Kaiser',
   'Institution': 'Google Brain',
   'Email': 'lukaszkaiser@google.com'},
  {'Name': 'Illia Polosukhin',
   'Institution': 'Google Research',
   'Email': 'illia.polosukhin@gmail.com'}],
 'AuthorContact': [{'Name': 'Ashish Vaswani',
   'Institution': 'Google Brain',
   'Email': 'avaswani@google.com'},
  {'Name': 'Noam Shazeer',
   'Institution': 'Google Brain',
   'Email': 'noam@google.com'},
  {'Name': 'Niki Parmar',
   'Institution': 'Google Research',
   'Email': 'niki

In [47]:
import ollama
import cntext as ct  
#cntext版本为2.1.2，非开源， #需联系大邓372335839获取

def extract_info(paper_content, prompt_content, max_retries=3):
    for attempt in range(max_retries + 1):
        try:
            response = ollama.chat(
                model='llama3.1:8b',
                messages=[
                    {'role': 'system', 'content': prompt_content},
                    {'role': 'user', 'content': paper_content}
                ]
            )

            result = response['message']['content']
            result = eval(result.split('```\n')[1].split('\n```')[0])
            return result
        
        except Exception as e:
            if attempt < max_retries:
                print(f"An error occurred: {e}. Retrying ({attempt + 1}/{max_retries + 1})...")
            else:
                raise e


#我们感兴趣的信息在论文的第一页，所以这里粗糙的选择前4000个字符。
paper_content = ct.read_pdf('data/2301.09056v1.pdf')[:4000]
prompt_content = open('prompts/scientific_papers_prompt.txt', encoding='utf-8').read()

result = extract_info(paper_content, prompt_content)
result

An error occurred: list index out of range. Retrying (1/4)...


{'PaperTitle': 'Performance Study of YOLO v5 and Faster R-CNN for Autonomous Navigation around Non-Cooperative Targets',
 'PublicationYear': 2022,
 'Authors': [{'Name': 'Trupti Mahendrakar'},
  {'Name': 'Andrew Ekblad'},
  {'Name': 'Nathan Fischer'},
  {'Name': 'Ryan T. White'},
  {'Name': 'Markus Wilde'},
  {'Name': 'Brian Kish'},
  {'Name': 'Isaac Silver'}],
 'AuthorContact': [{'Name': 'Trupti Mahendrakar',
   'Institution': 'Florida Institute of Technology',
   'Email': 'tmahendrakar2020@my.fit.edu'},
  {'Name': 'Andrew Ekblad',
   'Institution': 'Florida Institute of Technology',
   'Email': 'aekblad2019@my.fit.edu'},
  {'Name': 'Nathan Fischer',
   'Institution': 'Florida Institute of Technology',
   'Email': 'nfischer2018@my.fit.edu'},
  {'Name': 'Ryan T. White',
   'Institution': 'Florida Institute of Technology',
   'Email': 'rwhite@my.fit.edu'},
  {'Name': 'Markus Wilde',
   'Institution': 'Florida Institute of Technology',
   'Email': 'mwilde@fit.edu'},
  {'Name': 'Brian Kish

In [46]:
eval(result.split('```\n')[1].split('\n```')[0])

{'PaperTitle': 'Performance Study of YOLO v5 and Faster R-CNN for Autonomous Navigation around Non-Cooperative Targets',
 'PublicationYear': 2022,
 'Authors': [{'Name': 'Trupti Mahendrakar'},
  {'Name': 'Andrew Ekblad'},
  {'Name': 'Nathan Fischer'},
  {'Name': 'Ryan T. White'},
  {'Name': 'Markus Wilde'},
  {'Name': 'Brian Kish'},
  {'Name': 'Isaac Silver'}],
 'AuthorContact': [{'Name': 'Trupti Mahendrakar',
   'Institution': 'Florida Institute of Technology',
   'Email': 'tmahendrakar2020@my.fit.edu'},
  {'Name': 'Andrew Ekblad',
   'Institution': 'Florida Institute of Technology',
   'Email': 'aekblad2019@my.fit.edu'},
  {'Name': 'Nathan Fischer',
   'Institution': 'Florida Institute of Technology',
   'Email': 'nfischer2018@my.fit.edu'},
  {'Name': 'Ryan T. White',
   'Institution': 'Florida Institute of Technology',
   'Email': 'rwhite@my.fit.edu'},
  {'Name': 'Markus Wilde',
   'Institution': 'Florida Institute of Technology',
   'Email': 'mwilde@fit.edu'},
  {'Name': 'Brian Kish

In [48]:
%%time

import os
#cntext版本为2.1.3，非开源，需联系大邓372335839获取
import cntext as ct
import pandas as pd
import jsonlines

#当前代码所在的代码文件与data文件夹处于同一个文件夹内
#获取data内所有pdf的路径
pdf_files = [f'data/{file}' for file in os.listdir('data') if '.pdf' in file]
prompt_content = open('prompts/scientific_papers_prompt.txt', encoding='utf-8').read()

for pdf_file in pdf_files:
    paper_content = ct.read_pdf(pdf_file)[:4000]
    dict_data = extract_info(paper_content, prompt_content)
    jsonf = pdf_file.replace('data', 'extracted_metadata').replace('pdf', 'jsonl')
    with jsonlines.open(jsonf, 'w') as jf:
        jf.write(dict_data)

CPU times: user 919 ms, sys: 14.8 ms, total: 933 ms
Wall time: 24.6 s
